In [1]:
import importlib

import copy
from tabulate import tabulate

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [2]:
wildfire_fname = os.path.join("data","clean_fire_data.csv")
wildfire_table = MyPyTable()
wildfire_table.load_from_file(wildfire_fname)

fire_date = wildfire_table.get_column("date")
county = wildfire_table.get_column("county")
acres = wildfire_table.get_column("acres")
cause = wildfire_table.get_column("cause")
lat = wildfire_table.get_column("lat")
lng = wildfire_table.get_column("lon")

wildfire_X = [[fire_date[i], county[i], cause[i], lat[i], lng[i]] for i in range(len(fire_date))]
wildfire_y = [x for x in acres]





In [3]:
print(len(wildfire_X))
for i, x in enumerate(wildfire_X):
    if x[1] == "None" or x[1] == "NO DATA":
        del wildfire_X[i]
        del wildfire_y[i]
    elif x[2] == "None" or x[2] == "Logging" or x[2] == "Miscellaneou" or x[2] == "Under Invest" or x[2] == "Undetermined" or x[2] == "Railroad" or x[2] == "Arson":
        del wildfire_X[i]
        del wildfire_y[i]
print(len(wildfire_X))

58945
42206


In [4]:
rf = MyRandomForestClassifier()
rf.fit(wildfire_X, wildfire_y)

got sample
got validation set
28278 10263 73.37121506966606 26.62878493033393
got sample
got validation set
28278 10266 73.36550435865504 26.634495641344955
got sample
got validation set
28278 10200 73.49134570403866 26.50865429596133
got sample


KeyboardInterrupt: 

In [17]:
test_size = 10

X_train, X_test, y_train, y_test = myevaluation.train_test_split(copy.deepcopy(wildfire_X), copy.deepcopy(wildfire_y), test_size=test_size, shuffle=True)

nb = MyKNeighborsClassifier()
nb.fit(X_train, y_train)
predictions = []
for i, x in enumerate(X_test):
    sys.stdout.write("\r" + str(i) + "/" + str(len(X_test) -1) + "    ")
    sys.stdout.flush()
    prediction = nb.predict([x])
    predictions.append(prediction[0])

print()
acc = round(sum([int(x==y) for x,y in zip(predictions, y_test)])/len(predictions), 2)
print("KNN: accuracy = " + str(acc) + " error rate = " + str(1-acc))

9/9    
KNN: accuracy = 0.2 error rate = 0.8


In [18]:
print("getting training folds...")
X_train_folds, X_test_folds = myevaluation.stratified_kfold_cross_validation(wildfire_X, wildfire_y, n_splits=10)

getting training folds...


In [19]:
# k-fold stratified cross validation


print()
print("Stratified 10-Fold Cross Validation")

# decision trees for stratified k-fold
nb_predicted = []
nb_actual = []
# only does the first for now since it takes so long
for i in range(0, 1):
    X_train, X_test, y_train, y_test = myutils.get_trains_and_tests(wildfire_X, wildfire_y, X_train_folds[i], X_test_folds[i])
    nb = MyKNeighborsClassifier()
    nb.fit(X_train, y_train)
    predictions = []
    for i, x in enumerate(X_test):
        sys.stdout.write("\r" + str(i) + "/" + str(len(X_test) -1) + "    ")
        sys.stdout.flush()
        prediction = nb.predict([x])
        predictions.append(prediction[0])

    print()
    predicted = [ x for x in predictions]
    actual = [x for x in y_test]
    
    nb_predicted +=  predicted
    nb_actual += actual

acc = round(sum([int(x==y) for x,y in zip(predicted, actual)])/len(predicted), 2)
print("Naive Bayes: accuracy = " + str(acc) + " error rate = " + str(1-acc))


Stratified 10-Fold Cross Validation
4220/4220    
Naive Bayes: accuracy = 0.28 error rate = 0.72
